<a href="https://colab.research.google.com/github/ravimashru/END-assignments/blob/main/Assignment_9/05_seq2seq_cornell_movie_dialog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Tue Dec 29 05:56:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!cp drive/MyDrive/cornell-movie-dialog-turns.csv .

In [4]:
!ls -lah

total 24M
drwxr-xr-x 1 root root 4.0K Dec 29 05:56 .
drwxr-xr-x 1 root root 4.0K Dec 29 05:52 ..
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 .config
-rw------- 1 root root  24M Dec 29 05:56 cornell-movie-dialog-turns.csv
drwx------ 4 root root 4.0K Dec 29 05:54 drive
drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data


# Load data

In [5]:
import pandas as pd
import torchtext
import torch
import time
import random
import math
from tqdm.notebook import tqdm

from torch import nn, optim

In [6]:
df = pd.read_csv('cornell-movie-dialog-turns.csv')
df.head(50_000).to_csv('cornell-movie-dialog-turns-mini.csv')

In [7]:
print(df.shape)

(221616, 2)


In [8]:
# Use same field for both columns since they have a shared vocabulary
TEXT = torchtext.data.Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('turn1', TEXT), ('turn2', TEXT)]

In [9]:
# Create dataset
start = time.time()
dataset = torchtext.data.TabularDataset(
    path='cornell-movie-dialog-turns-mini.csv',
    format='CSV',
    fields=fields,
    skip_header=True
)
end = time.time()
print(f'Duration: {end - start}')

Duration: 6.801021575927734


In [10]:
# Train/val split
(train, valid) = dataset.split(split_ratio=[0.85, 0.15])
print(len(train), len(valid))

42500 7500


In [11]:
vars(train[0])

{'turn1': ['32000'],
 'turn2': ['you', 'know', 'i', "'m", 'a', 'good', 'person', '.']}

In [12]:
MAX_VOCAB_SIZE = 10_000

TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)

print(f'Size of vocab: {len(TEXT.vocab)}')

Size of vocab: 10004


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
BATCH_SIZE = 64

train_iterator, valid_iterator = torchtext.data.BucketIterator.splits(
    (train, valid),
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.turn1),
    device = device
)

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [18]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [19]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=10004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,763,092 trainable parameters


In [21]:
optimizer = optim.Adam(model.parameters())

In [22]:
TRG_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator)):
        
        src = batch.turn1
        trg = batch.turn2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in tqdm(enumerate(iterator)):

            src = batch.turn1
            trg = batch.turn2

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


Epoch: 01 | Time: 4m 26s
	Train Loss: 5.519 | Train PPL: 249.460
	 Val. Loss: 5.561 |  Val. PPL: 259.960



Epoch: 02 | Time: 4m 29s
	Train Loss: 5.144 | Train PPL: 171.458
	 Val. Loss: 5.549 |  Val. PPL: 256.996



Epoch: 03 | Time: 4m 25s
	Train Loss: 5.043 | Train PPL: 154.948
	 Val. Loss: 5.550 |  Val. PPL: 257.161



Epoch: 04 | Time: 4m 24s
	Train Loss: 4.981 | Train PPL: 145.589
	 Val. Loss: 5.544 |  Val. PPL: 255.799



Epoch: 05 | Time: 4m 24s
	Train Loss: 4.926 | Train PPL: 137.793
	 Val. Loss: 5.556 |  Val. PPL: 258.670



Epoch: 06 | Time: 4m 28s
	Train Loss: 4.911 | Train PPL: 135.746
	 Val. Loss: 5.539 |  Val. PPL: 254.523



Epoch: 07 | Time: 4m 26s
	Train Loss: 4.860 | Train PPL: 128.964
	 Val. Loss: 5.543 |  Val. PPL: 255.504



Epoch: 08 | Time: 4m 26s
	Train Loss: 4.815 | Train PPL: 123.297
	 Val. Loss: 5.556 |  Val. PPL: 258.761



Epoch: 09 | Time: 4m 25s
	Train Loss: 4.810 | Train PPL: 122.770
	 Val. Loss: 5.618 |  Val. PPL: 275.256



Epoch: 10 | Time: 4m 28s
	Train Loss: 4.766 | Train PPL: 117.408
	 Val. Loss: 5.566 |  Val. PPL: 261.366
